## Constant Volatility Fee Calculations

# ![Uniswap v3 fee tier formula](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*yah6u1x9uQkC7llZrz9QhQ.png)


In [105]:
import math

In [106]:
# feeTierDecimal = iv_per_year / (2 * math.sqrt(365 * 24 * 60 * 60)) * math.sqrt(tick_tvl / amount0) * math.sqrt(deltaT_secs)

def getFeeTierDecimal(iv_per_year, tick_tvl, amount0, deltaT_secs):
  two_sqrt_t = (2 * math.sqrt(365 * 24 * 60 * 60))
  # print('two_sqrt_t: ', two_sqrt_t) # > 11231.384598525687

  iv_seconds =  iv_per_year / two_sqrt_t
  # print('iv_seconds: {:.20f}'.format(iv_seconds))

  liq_swap_ratio = tick_tvl / amount0
  # print('liq_swap_ratio', liq_swap_ratio)

  sqrt_liq_swap_ratio = math.sqrt(liq_swap_ratio)
  # print('sqrt_liq_swap_ratio_t', sqrt_liq_swap_ratio)

  sqrt_delta_t = math.sqrt(deltaT_secs)
  # print('sqrt_delta_t', sqrt_delta_t)

  feeTier = iv_seconds * sqrt_liq_swap_ratio * sqrt_delta_t

  return feeTier


# return fee in ten thousandths (1 100th of a basis point)
def getFee(iv_per_year, tick_tvl, amount0, deltaT_secs):
  two_sqrt_t = 2 * math.sqrt(365 * 24 * 60 * 60)
  # print('two_sqrt_t: ', two_sqrt_t) # > 11231.384598525687

  iv_seconds =  iv_per_year / two_sqrt_t
  # print('iv_seconds: {:.20f}'.format(iv_seconds))

  liq_swap_ratio = tick_tvl / amount0
  # print('liq_swap_ratio', liq_swap_ratio)

  sqrt_liq_swap_ratio = math.sqrt(liq_swap_ratio)
  # print('sqrt_liq_swap_ratio_t', sqrt_liq_swap_ratio)

  sqrt_delta_t = math.sqrt(deltaT_secs)
  # print('sqrt_delta_t', sqrt_delta_t)

  feeTier = iv_seconds * sqrt_liq_swap_ratio * sqrt_delta_t

  return feeTier

In [107]:
# Example 1: 1 ETH trade with 100% annual volatility target
IV_per_year = 1_000_000  # 100% annual volatility, in fee units (1 / 100ths of a bip)
FEE_DECIMALS = 1_000_000
tick_tvl = 315     # Example TVL at tick
amount0 = 1        # 1 ETH trade
deltaT_secs = 15   # 1 block (15 seconds)

fee1Pct = getFeeTierDecimal(IV_per_year/FEE_DECIMALS, tick_tvl, amount0, deltaT_secs)
print(f"Fee (decimal) for 1 ETH trade after 1 block: {fee1Pct:.4}")
print(f"Fee (pct) for 1 ETH trade after 1 block: {fee1Pct:.4%}")
fee1 = getFee(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 1 block: {fee1:.10}")
print(f"Fee (to %) for 1 ETH trade after 1 block: {fee1 / 10_000:.10}%")
print('')
# > Fee for 1 ETH trade after 1 block: 0.6120%

deltaT_secs = 30   # 2 blocks
fee2Pct = getFeeTierDecimal(IV_per_year/FEE_DECIMALS, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 2 blocks: {fee2Pct:.4%}")
print(f"Fee (pct) for 1 ETH trade after 2 blocks: {fee2Pct:.4%}")
fee2 = getFee(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1 ETH trade after 2 blocks: {fee2:.10}")
print(f"Fee (to %) for 1 ETH trade after 2 blocks: {fee2 / 10_000:.10}%")
print('')
# > Fee for 1 ETH trade after 2 blocks: 0.8655%


# Example 3: 1000 ETH whale trade
amount0 = 1000     # 1000 ETH trade
deltaT_secs = 15   # 1 block
fee3Pct = getFeeTierDecimal(IV_per_year/FEE_DECIMALS, tick_tvl, amount0, deltaT_secs)
print(f"Fee (decimal) for 1000 ETH trade after 1 block: {fee3Pct:.4}")
print(f"Fee (pct) for 1000 ETH trade after 1 block: {fee3Pct:.4%}")
fee3 = getFee(IV_per_year, tick_tvl, amount0, deltaT_secs)
print(f"Fee for 1000 ETH trade after 1 block: {fee3:.10}")
print(f"Fee (to %) for 1000 ETH trade after 1 block: {fee3 / 10_000:.10}%")
print('')
# Fee for 1000 ETH whale trade: 0.0194%

# Total fees collected by LPs from whale trade
total_fees = amount0 * fee3Pct
print(f"Total fees collected from whale trade: {total_fees:.2f} ETH")
# Total fees collected from whale trade: 0.19 ETH

Fee (decimal) for 1 ETH trade after 1 block: 0.00612
Fee (pct) for 1 ETH trade after 1 block: 0.6120%
Fee for 1 ETH trade after 1 block: 6120.228082
Fee (to %) for 1 ETH trade after 1 block: 0.6120228082%

Fee for 1 ETH trade after 2 blocks: 0.8655%
Fee (pct) for 1 ETH trade after 2 blocks: 0.8655%
Fee for 1 ETH trade after 2 blocks: 8655.309559
Fee (to %) for 1 ETH trade after 2 blocks: 0.8655309559%

Fee (decimal) for 1000 ETH trade after 1 block: 0.0001935
Fee (pct) for 1000 ETH trade after 1 block: 0.0194%
Fee for 1000 ETH trade after 1 block: 193.5386054
Fee (to %) for 1000 ETH trade after 1 block: 0.01935386054%

Total fees collected from whale trade: 0.19 ETH
